In [1]:
import pormake as pm

In [2]:
db = pm.Database()
topo = db.get_topo("dia")
topo.describe()

Topology dia
Spacegroup: Fd-3m:2
-------------------------------------------------------------------------------
# of slots: 24 (8 nodes, 16 edges)
# of node types: 1
# of edge types: 1

-------------------------------------------------------------------------------
Node type information
-------------------------------------------------------------------------------
Node type: 0, CN: 4
  slot indices: 0, 1, 2, 3, 4, 5, 6, 7

-------------------------------------------------------------------------------
Edge type information (adjacent node types) 
-------------------------------------------------------------------------------
Edge type: (0, 0)
  slot indices: 8, 9, 10, 11, 12, 13, 14, 15, 16, 17
                18, 19, 20, 21, 22, 23


In [3]:
import numpy as np


def get_neighbor_nodes(topo, i):
    """
    Get indices and cell shifts of the neighbor nodes of the atom at index i.

    Parameters
    ----------
    topo : Topology
        The topology object.
    i : int
        The index of the atom.

    Returns
    -------
    nodes : list
        A list of tuples, each containing the index and cell shift of a neighbor node.
    """
    inv_cell = np.linalg.inv(topo.atoms.cell)
    edges = topo.neighbor_list[i]
    nodes = []
    for edge in edges:
        for node in topo.neighbor_list[edge.index]:
            if np.abs(node.distance_vector + edge.distance_vector).sum() < 1e-4:
                continue
            
            d = 2 * edge.distance_vector
            di = topo.atoms.positions[i]
            dj = topo.atoms.positions[node.index]
            # d = dj - di + c @ s.
            # s = inv_cell @ (dj - di - d)
            s = np.round(inv_cell @ (dj - di - d)).astype(int).tolist()
            nodes.append((int(node.index), tuple(s)))

    return sorted(nodes)

In [4]:
get_neighbor_nodes(topo, 0)

[(1, (0, 0, 1)), (3, (1, 0, 0)), (4, (1, 0, 1)), (7, (0, 0, 0))]

In [5]:
from collections import deque, defaultdict


def estimate_coordination_sequence(topo, i, max_level=10):
    """
    Estimate the coordination sequence of the atom at index i.

    Parameters
    ----------
    topo : Topology
        The topology object.
    i : int
        The index of the atom.
    max_level : int, optional
        The maximum level of the coordination sequence.

    Returns
    -------
    coordination_sequence : list
        The estimated coordination sequence.
    """
    q = deque([
        (i, (0, 0, 0), 0),
    ])
    visited = set([
        (i, (0, 0, 0))
    ])

    full_visited = set([
        (i, (0, 0, 0), 0)
    ])

    while q:
        node = q.popleft()
        
        if node[2] >= max_level:
            continue

        depth = node[2]
        nodes = get_neighbor_nodes(topo, node[0])
        new_nodes = []
        for n, s in nodes:
            new_s = (
                s[0] + node[1][0],
                s[1] + node[1][1],
                s[2] + node[1][2],
            )
            new_nodes.append((n, new_s, depth + 1))

        new_nodes = [n for n in new_nodes if n[:2] not in visited]
        q.extend(new_nodes)
        visited.update([(n, s) for n, s, _ in new_nodes])
        full_visited.update(new_nodes)

    count_depth = defaultdict(int)
    for n, s, d in list(full_visited):
        count_depth[d] += 1

    return [
        cs for _, cs in sorted(count_depth.items())[1:]
    ]


In [6]:
cs_list = []
for i in topo.node_indices:
    cs = estimate_coordination_sequence(topo, i)
    cs_list.append(cs)

    print(f"Estimated coordination sequence for atom {i}: {cs}")

Estimated coordination sequence for atom 0: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 1: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 2: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 3: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 4: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 5: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 6: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]
Estimated coordination sequence for atom 7: [4, 12, 24, 42, 64, 92, 124, 162, 204, 252]


In [7]:
cs_list = set([tuple(cs) for cs in cs_list])
cs_list = list(cs_list)
cs_list.sort()

# Unique key for a topology.
unique_key = tuple(cs_list)

print(unique_key)

((4, 12, 24, 42, 64, 92, 124, 162, 204, 252),)
